In [1]:
import sys
sys.path.append("./")
from collections import namedtuple
import mxnet as mx

data_dir="/_data"
customer = 'god'
zs_model_dir = '/models/'  # zero shot models
exp_out_dir = 'EXP'

il_model_dir = '/newmodels'

feat_names_file_init=os.path.join(zs_model_dir, '_prod.txt')
feat_names_file_stage_2 = feat_names_file_init
query_set = os.path.join(data_dir, customer + '/' + customer + '_query_sets.json')
dr_qrel = os.path.join(data_dir, customer + '/' + customer + '_test_dr_qrels.txt')
test_feedback = os.path.join(exp_out_dir, 'stage_1/' + customer + '_test_feedback.jsonl')
feat_names_stage_2 = os.path.join(zs_model_dir, '_prod.txt')
float_str = 'float16'
query_size = 50
nc_model_type = 'wx'
input_file = os.path.join(exp_out_dir,'inc_'+ str(query_size)+'/log'+nc_model_type+'_n_score_test.txt')
passage_file = os.path.join(exp_out_dir, 's/'+customer+'_passages.jsonl')

nc_model_stage_2=os.path.join(il_model_dir, customer+'/inc_learn_n_'+str(query_size)+'_n_best.params')
nc_model_init = os.path.join(zs_model_dir, 'best_nc_19.params')
args = namedtuple("args", "gpu log nc_model_file feat_names_file dr_batch_size query \
dr_ret_size nc batchnorm hiddens passages testfile dr_qrel input_file max_queries")
args.gpu = 0
args.log = 'log'
args.nc_model_file = nc_model_init
# args.nc_model_file = nc_model_stage_2

args.feat_names_file =  feat_names_file_init
args.dr_batch_size = 100
args.query = query_set
args.dr_ret_size = 1000
args.nc = 2
args.batchnorm = True
args.hiddens = [50, 10]
args.passages = passage_file
args.test_feedback = test_feedback
args.dr_qrel = dr_qrel 
args.input_file = input_file
args.max_queries = 0
with open(args.feat_names_file, 'r') as fp:
     feature_names = [line.strip('\n') for line in fp.readlines()]
meta_config = {
            'input_dim': len(feature_names),
            'nc': args.nc,
            'hiddens': args.hiddens,
            'batchnorm': args.batchnorm,
}

args.output_dir = os.path.join(exp_out_dir, 'inc_'+str(query_size))
proj_root = args.output_dir

log_file = os.path.join(proj_root, args.log)
ctx = mx.gpu(args.gpu)
args.use_mean_feature_val = False
predictor = exp(model_parameters=args.nc_model_file,
                                 feat_names_file=args.feat_names_file,
                                 meta_config=meta_config,
                                 use_mean_value=args.use_mean_feature_val,
                                 dr_batch_size=args.dr_batch_size,
                                 ret_size=args.dr_ret_size,
                                 ctx=ctx,
                                 float_str=float_str,
                                 log_file=log_file,
                                 )

score_dict = predictor.run( args.test_feedback, args.dr_qrel, max_queries=args.max_queries)


import csv
with open('./feedback.txt','w') as csv_file:
    for key in session.keys():
        csv_file.write("%s,%s\n"%(key, session[key]))

ModuleNotFoundError: No module named 'mxnet'